In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
! /opt/bin/nvidia-smi

/bin/bash: /opt/bin/nvidia-smi: No such file or directory


In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib
import gensim, time
import pickle, json
%matplotlib inline
from matplotlib import pyplot as plt

In [0]:
training_path = '/content/drive/My Drive/Colab Notebooks/project2/train.pkl'
validation_path = '/content/drive/My Drive/Colab Notebooks/project2/validation.pkl'
word2vec_path = '/content/drive/My Drive/Colab Notebooks/project2/cn.cbow.bin'
history_path = '/content/drive/My Drive/Colab Notebooks/project2/model/history.json'
model_path = '/content/drive/My Drive/Colab Notebooks/project2/model/model.tf'

batch_size = 8
truncate = 450
dimension = 300

tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu, steps_per_run=128) # Going back and forth between TPU and host is expensive. Better to run 128 batches on the TPU before reporting back.
print('Running on TPU ', tpu.cluster_spec().as_dict()['worker']) 

BATCH_SIZE = batch_size * strategy.num_replicas_in_sync # Gobal batch size.


INFO:tensorflow:Initializing the TPU system: 10.68.131.210:8470
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Querying Tensorflow master (grpc://10.68.131.210:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 13959921997667519874)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 12124004663629699873)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 10258986348942367726)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 8444659932386596494)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/d

In [0]:
train = pickle.load(open(training_path, 'rb'))
validation = pickle.load(open(validation_path, 'rb'))

In [0]:
columns = ['location_traffic_convenience',
   'location_distance_from_business_district', 'location_easy_to_find',
   'service_wait_time', 'service_waiters_attitude',
   'service_parking_convenience', 'service_serving_speed', 'price_level',
   'price_cost_effective', 'price_discount', 'environment_decoration',
   'environment_noise', 'environment_space', 'environment_cleaness',
   'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation',
   'others_overall_experience', 'others_willing_to_consume_again']
output = columns[0]

In [0]:
train = train[['content', output]]
train.columns = ['x', 'y']

In [0]:
validation = validation[['content', output]]
validation.columns = ['x', 'y']

In [0]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True, unicode_errors='ignore')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
x_train = train['x'].to_list()
y_train = train['y'].to_list()
x_validation = validation['x'].to_list()
y_validation = validation['y'].to_list()

In [0]:
x_train = [word2vec.wv[el]  for el in x_train]
x_validation = [word2vec.wv[el]  for el in x_validation]
del word2vec

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [0]:
class CIFAR10Sequence(tf.keras.utils.Sequence):
    def __init__(self, x, y, is_x):
      assert len(x) == len(y), '输入输出长度不匹配'
      self.x = x
      self.y = tf.keras.utils.to_categorical(y, 4)
      self.add = [np.zeros(300, dtype=np.float32)]
      self.truncate = 450
      self.dimension = 300
      self.is_x = is_x

    def __len__(self):
      return len(self.x)

    def __getitem__(self, idx):  # (inputs, targets)
      if self.is_x:
        x = self.x[idx]
        x = np.append(x, self.add * (self.truncate - x.shape[0]), axis=0) if x.shape[0] < self.truncate else x
        return x
      else:
        y = self.y[idx]
        return y

class CIFAR10Sequence_batch(tf.keras.utils.Sequence):
    def __init__(self, x, y, batch_size):
        self.x = x
        self.y = tf.keras.utils.to_categorical(y, 4)
        self.batch_size = batch_size
        self.add = np.zeros(300)
        
    def __len__(self):
        return len(self.x) // self.batch_size

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size: (idx + 1) * self.batch_size]
        batch_x = [np.append(x[:truncate], np.array([self.add for _ in range(truncate - len(x))]), axis=0) if len(x) < truncate else x[:truncate] for x in batch_x]
        batch_y = self.y[idx * self.batch_size: (idx + 1) * self.batch_size]
        return np.array(batch_x), np.array(batch_y)

train_x = CIFAR10Sequence(x_train, y_train, is_x=1)
train_y = CIFAR10Sequence(x_train, y_train, is_x=0)
validation_x = CIFAR10Sequence(x_validation, y_validation, is_x=1)
validation_y = CIFAR10Sequence(x_validation, y_validation, is_x=0)

train_x[0].shape, train_y[0].shape

((450, 300), (4,))

In [0]:
len(x_train)

105000

In [0]:
def gen(): 
  x, y = x_train, y_train
  add = [np.zeros(300, dtype=np.float32)]
  for i in range(len(x)):
    yield np.append(x[i], add * (truncate - x[i].shape[0]), axis=0) if x[i].shape[0] < truncate else x[i], y[i]

In [0]:
dataset = tf.data.Dataset.range(3) 
list(dataset.take(3)) 

RuntimeError: ignored

In [0]:
dataset = tf.data.Dataset.from_generator(gen, output_types=tf.float32)

In [0]:
list(dataset.take(3)) 

RuntimeError: ignored

INFO:tensorflow:Initializing the TPU system: 10.76.56.130:8470
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Querying Tensorflow master (grpc://10.76.56.130:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 7797236434981800356)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 4520922117299518275)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4539612141356078936)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 10894247709994612778)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/devic

In [0]:
def make_model():
  activation = 'relu'
  activation2 = 'relu'
  dropout = 0.5

  model = tf.keras.Sequential([
                              
      tf.keras.layers.Conv1D(filters=16, kernel_size=3, strides=1, input_shape=(truncate, dimension)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(activation),
      tf.keras.layers.MaxPooling1D(padding='same'),

      tf.keras.layers.Conv1D(filters=16, kernel_size=3, strides=1),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(activation),
      tf.keras.layers.MaxPooling1D(padding='same'),

      tf.keras.layers.Conv1D(filters=32, kernel_size=3, strides=1),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(activation),
      tf.keras.layers.MaxPooling1D(padding='same'),

      tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(activation),
      tf.keras.layers.MaxPooling1D(padding='same'),

      tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(activation),
      tf.keras.layers.MaxPooling1D(padding='same'),
      tf.keras.layers.Flatten(),

      tf.keras.layers.Dense(16),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(activation2),
      tf.keras.layers.Dropout(dropout),
      tf.keras.layers.Dense(8),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation(activation2),
      tf.keras.layers.Dropout(dropout),
      tf.keras.layers.Dense(4),
  ])
  
  lr = 0.01
  model.compile(optimizer=tf.keras.optimizers.SGD(lr),
                loss=tf.keras.losses.categorical_crossentropy,
                metrics = ['categorical_accuracy', 'AUC', 'Precision', 'Recall'])
  return model

In [0]:
with strategy.scope():
  model = make_model()
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 448, 16)           14416     
_________________________________________________________________
batch_normalization (BatchNo (None, 448, 16)           64        
_________________________________________________________________
activation (Activation)      (None, 448, 16)           0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 224, 16)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 222, 16)           784       
_________________________________________________________________
batch_normali

In [0]:
x_train = tuple(x_train)
y_train = tuple(y_train)

In [0]:
EPOCHS = 10
steps_per_epoch = len(train) // BATCH_SIZE
print("Steps per epoch: ", steps_per_epoch)

def gen(): 
  x, y = x_train, y_train
  add = [np.zeros(300, dtype=np.float32)]
  for i in range(len(x)):
    yield np.append(x[i], add * (truncate - x[i].shape[0]), axis=0) if x[i].shape[0] < truncate else x[i], y[i]

def x_gen(): 
  x, y = x_train, y_train
  add = [np.zeros(300, dtype=np.float32)]
  for i in range(len(x)):
    yield np.append(x[i], add * (truncate - x[i].shape[0]), axis=0) if x[i].shape[0] < truncate else x[i]

def y_gen(): 
  x, y = x_train, y_train
  for i in range(len(x)):
    yield y[i]

data = tf.data.Dataset.from_generator(gen, output_types=tf.float32, output_shapes=(450, 300))
# data = tf.data.Dataset.zip((x_train, y_train)) 
data = data.batch(BATCH_SIZE, drop_remainder=True) 
x = tf.data.Dataset.from_generator(x_gen, output_types=tf.float32, output_shapes=(450, 300))
y = tf.data.Dataset.from_generator(y_gen, output_types=tf.float32)
# data = tf.data.Dataset.zip((x, y)) 
# data = data.batch(BATCH_SIZE, drop_remainder=True) 

x_train[0].shape

Steps per epoch:  1640


(68, 300)

In [0]:
history = model.fit(data, epochs=EPOCHS, steps_per_epoch=steps_per_epoch)  # , batch_size=BATCH_SIZE

# final_stats = model.evaluate(validation, steps=1)
# print("Validation accuracy: ", final_stats[1])

ERROR:absl:Operation of type Placeholder (dense_2_target_1) is not supported on the TPU. Execution will fail if this op is used in the graph. 


Epoch 1/10


In [0]:
len(train)

105000

In [0]:
while 1:
  H = model.fit_generator(train, steps_per_epoch=len(train), epochs=1, validation_data=validation, validation_steps=len(validation), verbose=1)
  model.save(model_path)
  with open(history_path, 'a') as f:
    f.write(json.dumps({k:[float(el) for el in v] for k, v in dict(H.history).items()}) + '\n')
  print('model saved')

11331/13125 [========================>.....] - ETA: 25s - loss: 3.1944 - categorical_accuracy: 0.2029 - auc: 1.6502e-06 - precision: 0.2177 - recall: 0.2242

In [0]:
from_epoch = 10

loss = []
val_loss = []
with open('/content/drive/My Drive/Colab Notebooks/ai_price_predict/cnn_dense_A_history.json', 'r') as f:
  for line in f:
    line = json.loads(line)
    loss.extend(line['loss'])
    val_loss.extend(line['val_loss'])
assert len(loss) == len(val_loss), '检查json文件, 两个loss长度不一致'

N = np.arange(0, len(loss))
plt.style.use("ggplot")
plt.figure()
plt.plot(N[from_epoch:], loss[from_epoch:], label="train_loss")
plt.plot(N[from_epoch:], val_loss[from_epoch:], label="val_loss")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend()

In [0]:
plt.style.use("ggplot")
plt.figure()
plt.plot(N[from_epoch:], [i ** 0.5 for i in loss[from_epoch:]], label="train_mae")
plt.plot(N[from_epoch:], [i ** 0.5 for i in val_loss[from_epoch:]], label="val_mae")
plt.title("Training MAE")
plt.xlabel("Epoch #")
plt.ylabel("MAE")
plt.legend()